In [1]:
from robobrowser import RoboBrowser
from collections import namedtuple

import pandas as pd

In [2]:
browser = RoboBrowser(parser="html5lib")
with open("token", "r") as file:
    token = file.readline().strip()
communities = ['markforsterultras']
browser.session.cookies['login'] = token

In [3]:
MatchTip = namedtuple("MatchTip", "match_day match_id home_team away_team home_tip away_tip")

def build_tippabgabe_url(community, matchday):
    return f"https://www.kicktipp.de/{community}/tippabgabe?&spieltagIndex={matchday}"

def get_matches(browser, community, matchday):
    url = build_tippabgabe_url(community, matchday)
    browser.open(url)
    home_teams = [td.text for td in browser.select('tr > td:nth-child(2)')]
    away_teams = [td.text for td in browser.select('tr > td:nth-child(3)')]
    value_default_0 = lambda inp: int(inp.attrs["value"]) if "value" in inp.attrs else 0
    home_tips = [value_default_0(inp) for inp in browser.select('input[id$="_heimTipp"]')]
    away_tips = [value_default_0(inp) for inp in browser.select('input[id$="_gastTipp"]')]
    matches = []
    for i in range(len(home_teams)):
        try:
            matches.append(MatchTip(matchday, i, home_teams[i], away_teams[i], home_tips[i], away_tips[i]))
        except:
            print(f"Match is over {matchday=} {home_teams[i]=} {away_teams[i]=}")
    return matches


def print_match(match):
    print(f"{match.home_tip}:{match.away_tip}   {match.home_team} : {match.away_team}")

def make_tipps(browser, community, match_tips):
    tips_by_match_day = {}
    for t in match_tips:
        match_day = str(t.match_day)
        if match_day not in tips_by_match_day:
            tips_by_match_day[match_day] = []
        tips_by_match_day[match_day].append(t)
    
    for match_day, match_day_tips in tips_by_match_day.items():
        print(f"Placing bets for matchday {match_day}")
        browser.open(build_tippabgabe_url(community, match_day))
        form = browser.get_form()
        field_home_tips = browser.select('input[id$="_heimTipp"]')
        field_away_tips = browser.select('input[id$="_gastTipp"]')
        for match in match_day_tips:
            home_field = field_home_tips[match.match_id]
            away_field = field_away_tips[match.match_id]
            form[home_field.attrs['name']] = str(match.home_tip)
            form[away_field.attrs['name']] = str(match.away_tip)
            print(f"{match.home_tip}:{match.away_tip}   {match.home_team} : {match.away_team}")

        browser.submit_form(form, submit="submitbutton")

In [4]:
df_predictions = pd.read_csv("data/predictions_future.csv", index_col="id")
df_predictions = df_predictions[["match_day", "season", "host_id", "host_name", "guest_id", "guest_name", "host_goals_pred", "guest_goals_pred"]]
df_predictions.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/predictions_future.csv'

In [5]:
team_names_df = df_predictions["host_name"].unique().tolist()
team_names_df = [t.lower() for t in team_names_df]
team_names_df

['gladbach',
 'leipzig',
 'hoffenheim',
 'freiburg',
 'augsburg',
 'mainz',
 'bvb',
 'wolfsburg',
 'st. pauli',
 'union berlin',
 'stuttgart',
 'frankfurt',
 'bremen',
 'bochum',
 'kiel',
 'leverkusen',
 'heidenheim',
 'bayern']

In [6]:
from difflib import SequenceMatcher

def similar(names, b):
    return max([SequenceMatcher(None, n, b).ratio() for n in names])

def find_tid(name):
    name = name.lower()
    names = name.split(" ")
    if "dortmund" in names:
        names += "bvb"
    similarity_scores = [similar(names, t) for t in team_names_df]
    idxmax =  max(enumerate(similarity_scores),key=lambda x: x[1])[0]
    if similarity_scores[idxmax] < 0.5:
        print(f"Warning: Not sure if {name} really is {team_names_df[idxmax]}")
    team_name_df = team_names_df[idxmax]
    return df_predictions.loc[df_predictions["host_name"].str.lower() == team_name_df, "host_id"].iloc[0]


find_tid("Kiel")


np.int64(104)

In [7]:
def find_prediction(match):
    host_id = find_tid(match.home_team)
    guest_id = find_tid(match.away_team)
    row = df_predictions[(df_predictions["host_id"] == host_id) & (df_predictions["guest_id"] == guest_id)]
    home_tip = int(row["host_goals_pred"].iloc[0])
    away_tip = int(row["guest_goals_pred"].iloc[0])
    
    return MatchTip(match.match_day, match.match_id, match.home_team, match.away_team, home_tip, away_tip)

community_predictions = {}
for community in communities:
    all_predictions = []
    for match_day in range(1, 35):
        matches = get_matches(browser, community, match_day)
        predictions = [find_prediction(m) for m in matches]
        all_predictions += predictions
        make_tipps(browser, community, predictions)
    community_predictions[community] = all_predictions


Match is over matchday=1 home_teams[i]='Bor. Mönchengladbach' away_teams[i]='Bayer 04 Leverkusen'
Match is over matchday=1 home_teams[i]='1899 Hoffenheim' away_teams[i]='Holstein Kiel'
Match is over matchday=1 home_teams[i]='RB Leipzig' away_teams[i]='VfL Bochum'
Match is over matchday=1 home_teams[i]='FC Augsburg' away_teams[i]='Werder Bremen'
Match is over matchday=1 home_teams[i]='SC Freiburg' away_teams[i]='VfB Stuttgart'
Match is over matchday=1 home_teams[i]='FSV Mainz 05' away_teams[i]='1. FC Union Berlin'
Match is over matchday=1 home_teams[i]='Borussia Dortmund' away_teams[i]='Eintracht Frankfurt'
Match is over matchday=1 home_teams[i]='VfL Wolfsburg' away_teams[i]='FC Bayern München'
Match is over matchday=1 home_teams[i]='FC St. Pauli' away_teams[i]='1. FC Heidenheim 1846'
Match is over matchday=2 home_teams[i]='1. FC Union Berlin' away_teams[i]='FC St. Pauli'
Match is over matchday=2 home_teams[i]='VfB Stuttgart' away_teams[i]='FSV Mainz 05'
Match is over matchday=2 home_te